In [2]:
#%matplotlib notebook
import glob
#import io
import os
import cv2
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from svglib.svglib import svg2rlg
from collections import defaultdict
from fiona.crs import from_epsg
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
import numpy as np
#import matplotlib; matplotlib.use('agg')
#import matplotlib.pyplot as plt
#from matplotlib.figure import Figure
#from matplotlib.backends.backend_agg import FigureCanvas

from archigan.datalayer import Layer

In [3]:
class FootprintInputLayer(Layer):
    
    labels = dict([
        ('Footprint', 5),
    ])

    def __call__(self, byclass, height, width):
        mask = self.norm_mask(self.mask(byclass, height, width))
        layer = mask.max() - mask
        return layer

        
class RepartitionInputLayer(Layer):
    
    labels = dict([
        ('Door', 1),
        ('Window', 2),
        ('Parking', 3),
        ('Room', 5),
    ])

    def __call__(self, byclass, height, width):
        # start with the inverse of the mask
        # blend in annotations which touch the boundary of the footprint
        mask = self.norm_mask(self.mask(byclass, height, width))
        layer = mask.max() - mask
        for cls, label in self.labels.items():
            for py in byclass[cls]:
                # determine if a dilated mask of py intersects the mask
                # which implies its a feature on the boundary of the footprint
                pymask = self.mask({cls: [py]}, height, width)
                kernel = np.ones((3, 3), np.uint8) 
                pyfilt = cv2.erode(pymask.copy(), kernel, iterations=1)
                pyfilt = (((mask) * (pyfilt.max() - pyfilt)).max() > 0)
                if pyfilt or cls == 'Room':
                    weight = (pymask == 0).astype(int)
                    layer = weight * label + (1 - weight) * layer
        return layer


class RepartitionOutputLayer(Layer):
    
    labels = dict([
        ('Door', 1),
        ('Window', 2),
        ('Parking', 3),
        ('Room', 4),
        ('Wall', 5),
    ])

    def __call__(self, byclass, height, width):
        mask = self.norm_mask(self.mask(byclass, height, width))
        layer = mask.max() - mask
        # start with the inverse of the mask
        # blend in annotations which touch the boundary of the footprint
        for cls, label in self.labels.items():
            for py in byclass[cls]:
                # determine if a dilated mask of py intersects the mask
                # which implies its a feature on the boundary of the footprint
                pymask = self.mask({cls: [py]}, height, width)
                kernel = np.ones((3, 3), np.uint8) 
                pymask = cv2.erode(pymask, kernel, iterations=3)
                # blend a mask of py with the layer using a binary weight
                weight = (pymask == 0).astype(int)
                layer = weight * label + (1 - weight) * layer
        return layer

In [4]:
def parse_CVC_FP_svg(path, classes):
    """Parser for SVGs form the CVC-FP dataset:
        http://dag.cvc.uab.es/resources/floorplans/
    """
    drawing = svg2rlg(path)
    byclass = defaultdict(list)
    for cls in classes:
        for py in drawing.contents[0].contents:
            if py._class == cls:
                loop = list(zip(py.points[::2], py.points[1::2]))
                byclass[py._class].append(loop)
    sample = {
        'byclass': byclass,
        'height': drawing.height,
        'width': drawing.width,
    }
    return sample

In [5]:
def parse_parcel(py):
    """Parse the Polygon/MultiPolygon objects from shp files"""
    if isinstance(py, Polygon):
        bound = list(zip(*py.exterior.coords.xy))
        holes = [list(zip(*hole.coords.xy)) for hole in py.interiors]
        #return bound, holes
        return [bound]
    elif isinstance(py, MultiPolygon):
        parts = [parse_parcel(part) for part in py.geoms]
        return [x for y in parts for x in y]
    else:
        print('bad shape type:', type(py))

def parse_building(py):
    return parse_parcel(py)
    
def bbox(pts):
    """Find the AABB bounding box for a set of points"""
    x, y = pts[0]
    ax, ay, bx, by = x, y, x, y
    for i in range(1, len(pts)):
        x, y = pts[i]
        ax = x if x < ax else ax
        ay = y if y < ay else ay
        bx = x if x > bx else bx
        by = y if y > by else by
    return ax, ay, bx, by

def to_first_quadrant(parcel, building):
    """Translate parcel and building to the first quadrant"""
    pts = [p for part in (parcel + building) for p in part]
    ax, ay, bx, by = bbox(pts)
    height, width = (bx - ax), (by - ay)
    parcel = [[(x - ax, y - ay) for x, y in part] for part in parcel]
    building = [[(x - ax, y - ay) for x, y in part] for part in building]
    return parcel, building, height, width

def parse_GIS_bostonbuildings_2016(path):
    """Parser for 2016 boston buildings GIS data:
        https://www.arcgis.com/home/item.html?id=c423eda7a64b49c98a9ebdf5a6b7e135
        https://data.boston.gov/dataset/parcels-2016-data-full/resource/d53d8e93-034d-4dd0-b59f-8634f4df3a71    
    """
    # read in GIS data via geopandas
    parcels = os.path.join(path, 'Parcels_2016_Data_Full.shp')
    print(f'Reading parcel data: {parcels}')
    parcels = gpd.read_file(parcels)
    buildings = os.path.join(path, 'boston_buildings.shp')
    print(f'Reading footprint data: {buildings}')
    buildings = gpd.read_file(buildings)
    # reproject buildings using parcels' CRS
    epsg = parcels.crs.to_epsg()
    print(f'Reprojecting footprint data to EPSG: {epsg}')
    buildings['geometry'] = buildings['geometry'].to_crs(epsg=epsg)
    buildings.crs = from_epsg(epsg)
    # create samples of complete pairs of parcel/building outlines
    pidlookup = {row.PID: j for j, row in parcels.iterrows()}
    samples = []
    for j, row in tqdm(buildings.iterrows(), desc='Preparing parcel/footprint data'):
        if row.PARCEL_ID in pidlookup:
            parcel = parse_parcel(parcels.iloc[pidlookup[row.PARCEL_ID]].geometry)
            building = parse_parcel(row.geometry)
            parcel, building, height, width = to_first_quadrant(parcel, building)
            sample = {
                'byclass': {
                    'parcel': parcel,
                    'building': building,
                },
                'height': height,
                'width': width,
            }
            samples.append(sample)
    return samples

In [6]:
class ParcelInputLayer(Layer):
    
    labels = dict([
        ('parcel', 5),
    ])

    def parcel_mask(self, byclass, height, width):
        margin = 0.1 * max(height, width)
        parcel = {'parcel': byclass['parcel']}
        mask = self.mask(parcel, height, width, margin=margin)
        # erode so that footprints are better contained by parcels on average...
        kernel = np.ones((3, 3), np.uint8)
        layer = cv2.erode(mask, kernel, iterations=3)
        layer = self.norm_mask(layer)
        return self.norm_mask(layer)
    
    def __call__(self, byclass, height, width):
        layer = self.parcel_mask(byclass, height, width)
        layer = layer.max() - layer
        return layer

class ParcelOutputLayer(ParcelInputLayer):
    
    labels = dict([
        ('parcel', 5),
        ('building', 3),
    ])

    def __call__(self, byclass, height, width):
        layer = self.parcel_mask(byclass, height, width)
        layer = layer.max() - layer
        # blend in building footprints
        for cls, label in self.labels.items():
            for py in byclass[cls]:
                
                ## determine if a dilated mask of py intersects the mask
                ## which implies its a feature on the boundary of the footprint
                margin = 0.1 * max(height, width) # ugh that this is in 2 places...
                pymask = self.mask({cls: [py]}, height, width, margin=margin)

                kernel = np.ones((3, 3), np.uint8) 
                pymask = cv2.erode(pymask.copy(), kernel, iterations=3)

                weight = (pymask == 0).astype(int)
                layer = weight * label + (1 - weight) * layer
        return layer

In [9]:
path = 'datasets/boston_buildings/2016'
parsed = parse_GIS_bostonbuildings_2016(path)
parsed = {f'boston_{j}': sample for j, sample in enumerate(parsed[:100])}
directory = './tmp4/'

print('N:', len(parsed))

ParcelInputLayer.samples_to_imgs(parsed, directory)
ParcelOutputLayer.samples_to_imgs(parsed, directory)

Reading parcel data: datasets/boston_buildings/2016/Parcels_2016_Data_Full.shp
Reading footprint data: datasets/boston_buildings/2016/boston_buildings.shp
Reprojecting footprint data to EPSG: 3857



N: 100


In [10]:
!pwd

/home/cogle/dev/mine/archigan


In [8]:
svgs = glob.glob('datasets/ImagesGT/*.svg')
classes = (
    'Door',
    'Window',
    'Room',
    'Wall',
    'Separation',
    'Parking',
)
parsed = {svg: parse_CVC_FP_svg(svg, classes) for svg in svgs[3:30]}
directory = './tmp4/'

FootprintInputLayer.samples_to_imgs(parsed, directory)
RepartitionInputLayer.samples_to_imgs(parsed, directory)
RepartitionOutputLayer.samples_to_imgs(parsed, directory)

In [7]:
import geopandas as gpd

df = gpd.read_file('datasets/boston_streets/Boston_Street_Segments.shp')
df.head()

,OBJECTID,SEGMENT_ID,L_F_ADD,L_T_ADD,R_F_ADD,R_T_ADD,PRE_DIR,ST_NAME,ST_TYPE,SUF_DIR,...,PLACE00_R,TRACT00_L,TRACT00_R,BLOCK00_L,BLOCK00_R,MCD00_L,MCD00_R,STREET_ID,SHAPElen,geometry
0,1,1.0,12,22,13,29,None,A,ST,None,...,07000,02127,02127,None,None,07000,07000,2.0,166.673187,"LINESTRING (776005.375 2949518.000, 776115.625..."
1,2,2.0,24,42,31,43,None,A,ST,None,...,07000,02127,02127,None,None,07000,07000,2.0,162.142997,"LINESTRING (776115.625 2949643.000, 776219.501..."
2,3,3.0,44,64,45,67,None,A,ST,None,...,07000,02127,02127,None,None,07000,07000,2.0,178.522006,"LINESTRING (776219.501 2949767.499, 776336.312..."
3,4,4.0,66,72,69,71,None,A,ST,None,...,07000,02127,02127,None,None,07000,07000,2.0,196.841018,"LINESTRING (776336.312 2949902.500, 776466.375..."
4,5,5.0,74,90,73,87,None,A,ST,None,...,07000,02127,02127,None,None,07000,07000,2.0,179.277949,"LINESTRING (776466.375 2950050.250, 776584.622..."
